In [27]:
import os

# Ignore numpy warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
#%matplotlib notebook

# Use seaborn settings.
import seaborn as sns
sns.set(
        #context="paper",
        context="talk",
        #context="poster",
        style='darkgrid',
        #style="dark",
        #palette='deep',
        font='sans-serif', 
        #font_scale=1.0, 
        #color_codes=False, 
        rc={'figure.figsize': (12, 8.0)}, # width and height in inches.
)

import IPython

def _embed(src, width="800", height="400"):
    return IPython.display.IFrame(src=src, width=str(width), height=str(height))

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def embed(line):
    "my line magic"
    #return line
    return _embed(line)


# We delete these to avoid name conflicts for automagic to work
del embed
#del lcmagic

# Theme
# import jtplot module in notebook
#from jupyterthemes import jtplot

# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
#jtplot.style(theme='monokai')

# set "context" (paper, notebook, talk, poster)
# scale font-size of ticklabels, legend, etc.
# remove spines from x and y axes and make grid dashed
#jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')

# turn on X- and Y-axis tick marks (default=False)
# turn off the axis grid lines (default=True)
# and set the default figure size
#jtplot.style(ticks=True, grid=False, figsize=(6, 4.5))

# reset default matplotlib rcParams
#jtplot.reset()

from abipy import abilab
import abipy.data as abidata

si_pseudo = os.path.join(abidata.pseudo_dir, "si.psp8")

# Technical aspects related to automatic calculations with ABINIT

### M. Giantomassi and the AbiPy group

9th international ABINIT developer workshop<br>
<small>20-22nd May 2019 - Louvain-la-Neuve, Belgium</small>

<!-- link rel="stylesheet" href="reveal.js/css/theme/sky.css" id="theme" -->

<img src="./assets/intro_logos.png" width="85%" heigh="15%" align="center">

## Overview:

* Approach used to connect AbiPy with Abinit
* Advantages and drawbacks of Yaml and *netcdf*
* Technical problems related to automatic calculations and possible solutions

<!--
<hr> 

* These slides have been generated using [jupyter](https://jupyter.org), [nbconvert](https://github.com/jupyter/nbconvert) and [revealjs](https://revealjs.com/)

* The notebook can be downloaded from this [github repo](https://github.com/gmatteo/abipy_htc_technical_aspects_abidev2019)

* To install and configure the software, follow these [installation instructions](https://github.com/abinit/abipy#getting-abipy)

Use the Space key to navigate through all slides.


In this talk, I will present the approach employed in AbiPy to implement automatic ABINIT calculations. More specifically, I will discuss how we employ
YAML and netcdf files to exchange information between Fortran and Python,
the logic used in AbiPy to parallelize and optimize calculations at runtime and
the protocol used to handle possible errors during the ab-initio computation. I
will also discuss how to use the Fortran API of ABINIT to generate netcdf files
that can interoperate seamlessly with our high-throughput infrastructure. In the
last part, I will present some of the technical problems we are still facing and
discuss possible approaches to address these issues.
-->

<img src="./assets/abipy_logo.jpg" width="55%" align="center">

## What is AbiPy?

#### Python package for:

   * Generating ABINIT input files automatically
   * Post-processing output results (*netcdf* and *text* files)
   * Interfacing ABINIT with external tools (e.g. [Vesta](http://jp-minerals.org/vesta/en/))
   * Creating and executing workflows (band structures, phonons, $GW$…)
   
#### Project:

* Developed and maintained by the ABINIT community
* Used by developers to validate, profile and optimize ABINIT
* Hosted on [github](https://github.com/abinit/abipy) 
* Release under the GPLv2 license

## What do we need to automate calculations?

* Tools to parse and analyze output results
* Programatic interface to generate input files
* High-level logic for:
    * Managing complicated workflows
    * Exposing task parallelism (independent steps can be executed in parallel)
    * Handling runtime errors and restarting calculations
    * Saving final results in machine-readable format (e.g. databases)
<p></p>
<hr>

<center>
<img src="./assets/sdata201865-f2.jpg" width="65%" align="center">
</center>

## AbiPy design principles

* Extend the pymatgen code-base with ABINIT-specific objects
* Layered structure designed for different use-cases:
        
     * Post-processing tools and command-line interfaces
     * API to automate calculations and data analysis
     * High-throughput infrastructure ([abiflows](https://github.com/abinit/abiflows), [fireworks](https://github.com/materialsproject/fireworks), [mongodb](https://www.mongodb.com/))
     
* Closely connected to the ABINIT executable:

    * CPU-intensive algorithms performed by ABINIT (Fortran + MPI + OpenMP)
    * Use files in machine-readable format to exchange data
    * High-level logic implemented in python
    * Can invoke Abinit directly from AbinitInput instance

## Workflow infrastructure

Two different approaches:

#### AbiPy workflows (*flowtk*):

   * Lightweight implementation (*pymatgen* + *AbiPy*)
   * No database required. Object persistence provided by [*pickle*](https://docs.python.org/3/library/pickle.html)
   * Ideal tool for prototyping, testing, debugging.
    
    
#### AbiFlows workflows: 

   * Based on [fireworks](https://materialsproject.github.io/fireworks/)
   * Requires [MongoDb](https://www.mongodb.com/) database
   * Designed and optimized for <u>high-throughput applications</u> (Guido's talk)
   
<hr>

Both approaches share the same codebase (*AbinitInput*, factory functions, AbiPy objects, TaskManager...).

Number and type of calculations are important ➝ choose according to your needs.

## Main features of workflows

* Factory functions for typical calculations (DFPT, GW, BSE, EPH, ...)
* Support for different resource managers (*Slurm*, *PBS*, *Torque*, *SGE*, *LoadLever*, *shell*)
* *autoparal*: the number of MPI processes is optimized at runtime
* Iterative algorithms are automatically restarted by the framework
* Error handlers for common runtime failures
<hr>
<p></p>

<center>
<img src="./assets/not_converged_task_2.png" width="40%" align="center">
</center>

## *AbinitInput* object 

Programmatic interface to generate input files:
* *Dict-like* object storing ABINIT variables 
* Methods to set multiple variables (*e.g.* **k**-path from *structure*)
* Factory functions to generate input files with minimal effort

Can invoke ABINIT to get important parameters such as:
  * list of **k**-points in the IBZ
  * list of irreducible perturbations for DFPT
  * list of possible configurations for MPI jobs (*npkpt*, *npfft*, *npband* …)
     
#### To build an input, we need a *structure* and a list of *pseudos*:

In [28]:
inp = abilab.AbinitInput(structure="si.cif", pseudos="si.psp8")

#### Low-level API (should look familiar to Abinit users):

In [29]:
inp["ecut"] = 8
"ecut" in inp

True

#### Use *set_vars* to set the value of several variables with a single call:

In [30]:
inp.set_vars(kptopt=1, ngkpt=[2, 2, 2],  
             shiftk=[0.0, 0.0, 0.0, 0.5, 0.5, 0.5]  # 2 shifts in one list
            );

#### althought it's much easier to use:

In [31]:
inp.set_autokmesh(nksmall=2)

{'ngkpt': array([2, 2, 2]),
 'kptopt': 1,
 'nshiftk': 4,
 'shiftk': array([[0.5, 0.5, 0.5],
        [0.5, 0. , 0. ],
        [0. , 0.5, 0. ],
        [0. , 0. , 0.5]])}

####  The method builds a homogeneous k-mesh from metavariables:

* *nksmall* is the number of divisions used to sample the smallest lattice vector
* *shiftk* is automatically selected from an internal database.

#### An *AbinitInput* has a structure object 

In [32]:
print(inp.structure)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.866975   3.866975   3.866975
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25


#### and a list of pseudopotentials

In [33]:
print(inp.pseudos[0])

<NcAbinitPseudo: si.psp8>
  summary: Si    ONCVPSP-3.2.3.1  r_core=   1.60303   1.72197   1.91712
  number of valence electrons: 4.0
  maximum angular momentum: d
  angular momentum for local part: g
  XC correlation: PBE
  supports spin-orbit: False
  radius for non-linear core correction: 0.0
  hint for low accuracy: ecut: 0.0, pawecutdg: 0.0
  hint for normal accuracy: ecut: 0.0, pawecutdg: 0.0
  hint for high accuracy: ecut: 0.0, pawecutdg: 0.0


## Interfacing Abinit with Python via *AbinitInput*

* Once we have an *AbinitInput*, it is possible to execute Abinit to:

    * get useful data from the Fortran code (e.g. IBZ, space group…)
    * validate the input file before running the calculation 
        
* Methods invoking Abinit start with the *abi* prefix followed by a verb: 

    * *inp.abiget_irred_phperts(...)* 
    * *inp.abivalidate()*

#### Important: 

* To call Abinit from AbiPy, one has to prepare a configuration file (*manager.yml*)
  providing all the information required to execute/submit Abinit jobs: 

     * `$PATH`, `$LD_LIBRARY_PATH`
     * modules (if any)

* For futher info consult the [documentation](https://abinit.github.io/abipy/workflows/taskmanager.html)

## Example of *manager.yml* for laptops (shell adapter)
<hr>

```yaml

qadapters:
  # List of qadapters objects 
  - priority: 1 
    queue:
      qtype: shell
      qname: localhost
    job:
       mpi_runner: mpirun
       pre_run: 
           # abinit exec must be in $PATH
           - export PATH=$HOME/git_repos/abinit/_build/src/98_main:$PATH
    limits:
       timelimit: 30:00
       max_cores: 2
    hardware:
       num_nodes: 1
       sockets_per_node: 1
       cores_per_socket: 2
       mem_per_node: 4Gb
```

* Examples of configuration files for clusters available [here](https://abinit.github.io/abipy/workflows/manager_examples.html)

* Use *abirun.py doc_manager* to get documentation inside the shell

## An example for a Slurm-based cluster
<hr>

```yaml

hardware: &hardware
   num_nodes: 80
   sockets_per_node: 2
   cores_per_socket: 12
   mem_per_node: 95Gb

job: &job
    mpi_runner: mpirun
    modules: # Load modules used to compile Abinit 
        - intel/2017b
        - netCDF-Fortran/4.4.4-intel-2017b
        - abinit_8.11
    pre_run: "ulimit -s unlimited"

# Slurm options.
qadapters:
  - priority: 1
    queue:
       qtype: slurm
       qname: large
    limits:
       timelimit: 0-0:30:00
       min_cores: 1
       max_cores: 48
       min_mem_per_proc: 1000
       max_mem_per_proc: 2000
       max_num_launches: 10
    hardware: *hardware
    job: *job
```

#### To get the list of k-points in the IBZ as computed by Abinit:

In [34]:
ibz = inp.abiget_ibz()

print("Number of k-points:", len(ibz.points))
print("Weights normalized to:", ibz.weights.sum())

n = min(5, len(ibz.points))
for i, (k, w) in enumerate(zip(ibz.points[:n], ibz.weights[:n])):
    print(i, "kpt:", k, "weight:", w)
if n != len(ibz.points): print("...")

Number of k-points: 2
Weights normalized to: 1.0
0 kpt: [-0.25  0.5   0.  ] weight: 0.75
1 kpt: [-0.25  0.    0.  ] weight: 0.25


* Being able to call ABINIT from python is crucial for implementing automatic calculations,
  especially when symmetries and $\bf{k}$-points are involved
* The calculation is done in Fortran so ABINIT is always right even when it's wrong!

#### To get the list of possible parallel configurations for this input up to *max_ncpus*:

In [35]:
inp["paral_kgb"] = 1
pconfs = inp.abiget_autoparal_pconfs(max_ncpus=5)
print("Best efficiency:\n", pconfs.sort_by_efficiency()[0])
#print("Best speedup:\n", pconfs.sort_by_speedup()[0])

Best efficiency:
 {'efficiency': 0.788,
 'mem_per_cpu': 0.0,
 'mpi_ncpus': 2,
 'omp_ncpus': 1,
 'tot_ncpus': 2,
 'vars': {'bandpp': 1,
          'npband': 1,
          'npfft': 1,
          'npimage': 1,
          'npkpt': 2,
          'npspinor': 1}}



* python will select the "optimal" configuration according to some criterion and additional constraints 
  specified by the user *e.g.* exclusive node allocation
  
<!--
* Crucial for implementing automatic $GW$ calculations but...
* Supporting *autoparal* for all possible cases is not trivial. 
* For new developments (e.g. EPH) we prefer to implement smart logic in fortran 
-->

#### To get the list of irreducible phonon perturbations for a given q-point:

In [36]:
inp.abiget_irred_phperts(qpt=(0.25, 0, 0))

[{'qpt': [0.25, 0.0, 0.0], 'ipert': 1, 'idir': 1},
 {'qpt': [0.25, 0.0, 0.0], 'ipert': 1, 'idir': 2}]

#### To get the irreducible perturbations for strain calculations:

In [37]:
inp.abiget_irred_strainperts()

[{'qpt': [0.0, 0.0, 0.0], 'ipert': 1, 'idir': 1},
 {'qpt': [0.0, 0.0, 0.0], 'ipert': 5, 'idir': 1},
 {'qpt': [0.0, 0.0, 0.0], 'ipert': 5, 'idir': 2},
 {'qpt': [0.0, 0.0, 0.0], 'ipert': 5, 'idir': 3},
 {'qpt': [0.0, 0.0, 0.0], 'ipert': 6, 'idir': 1},
 {'qpt': [0.0, 0.0, 0.0], 'ipert': 6, 'idir': 2},
 {'qpt': [0.0, 0.0, 0.0], 'ipert': 6, 'idir': 3}]

* DFPT perturbations are independent hence jobs can be executed in parallel
* Handling errors in calculations with datasets is not trivial ➔ Divide and conquer approach
* These methods represent the **building block** to generate workflows at runtime.
<!-- div class="alert alert-info"> </div -->

<center><img src="./assets/how_does_it_work.jpg" width="45%" align="center"></center>

* ABINIT and AbiPy communicate through Yaml docs in *log* and *netcdf* files
* AbiPy uses this protocol to:
     * generate input files
     * handle errors
     * implement high-level logic
* Config options required to run/submit calculations are specified in *manager.yml* 
* Programmatic interface to set number of MPI procs and job parameters

## Why two formats?

Each format has pros and cons:

* **YAML**
    * Mainly used for small documents:
        * error messages in log file
        * *autoparal* section and list of DFPT *perturbations* 
    * Easy to write with plain Fortran write (provided doc is *flat*) 
    * Not designed for big-data and performance  
    * Yaml should be used when both human- and machine-readability are wanted (*cfr.* Theo's talk)
        
* **Netcdf4**
    * Portable binary format. Can handle big-data and <u>metadata</u> efficiently with MPI-IO
    * [ETSF-IO specifications](https://www.etsf.eu/fileformats) for crystalline structures, wavefunctions, densities…
    * Still, ETSF-IO specs are not enough as we need *hdr_t* to restart. Besides many physical properties 
      are not covered by the standard
    * A machine-readable format is also human-readable. Just put some python code around it!

## Error messages in Yaml
<p></p>

#### Fortran API
<p></p>

```fortran
MSG_ERROR("Fatal error!")
MSG_BUG("This is a bug!")
ABI_CHECK(natom > 0, sjoin("Input natom must be > 0 but was:", itoa(natom))
```

* Macros to call the error handler and MPI_ABORT to terminates all processes 
* Never ever use plain Fortran *stop* in MPI codes 

#### Yaml doc printed to log file and  *__ABI_ABORT_FILE__*
<p></p>

```yaml
--- !ERROR
src_file: m_invars1.F90
src_line: 314
mpi_rank: 0
message: |
    Input natom must be > 0, but was -42 
...
```

* *src_file* and *str_line* added by CPP (see *abi_common.h*)
* *message*: String meant for users. It might be used by AbiPy for quick fixes 
  although we usually prefer a more robust approach...

## Specialized error classes 

* Runtime errors appearing with a certain frequency are associated to specialized error classes:  
<p></p>

```fortran
MSG_ERROR_CLASS(dilatmx_errmsg, "DilatmxError")
```

* These classes bring metadata and/or imply some action at the Fortran level 

* The presence of the Yaml error in the *log* file triggers specialized python logic:

<center><img src="./assets/dilatmx_2.png" width="30%" align="center"></center>

* As usual, error handling requires some discipline 🧘...

## Fortran handler based on YAML + netcdf
<p></p>

```fortran
call chkdilatmx(dt_chkdilatmx, dilatmx, rprimd, rprimd_orig, dilatmx_errmsg)

if (len_trim(dilatmx_errmsg) /= 0) then
    ! Write last structure before aborting so that we can restart from it.
    if (my_rank == master) then
      NCF_CHECK(crystal%ncwrite_path("out_DILATMX_STRUCT.nc")))
    end if
    call xmpi_barrier(comm_cell)
    write(dilatmx_errmsg, '(a,i0,3a)') &
      'Dilatmx has been exceeded too many times (', nerr_dilatmx, ')',ch10, &
      'Restart calculation from larger lattice vectors and/or a larger dilatmx'
    MSG_ERROR_CLASS(dilatmx_errmsg, "DilatmxError")
end if
```

* If `!DilatmxError` in *log* file, AbiPy expects a *ncfile* with the last structure
* Note MPI_BARRIER to guarantee that the *ncfile* is written before MPI_ABORT

## How do we understand that the job completed successfully?
<p></p>

```yaml
--- !FinalSummary
program: abinit
version: 8.11.6
start_datetime: Sat Mar 30 23:01:14 2019
end_datetime: Sat Mar 30 23:04:04 2019
overall_cpu_time:         168.8
overall_wall_time:         169.7
exit_requested_by_user: no 
timelimit: 0
pseudos: 
    Li  : 9517c0b7d24d4898578b8627ce68311d
    F   : 14cf65a61ba7320a86892d2f062b1f44
usepaw: 0
mpi_procs: 1
omp_threads: 1
num_warnings: 2
num_comments: 73
...
```

* Understanding whether the job is still running or stuck is more difficult since there's no "official protocol" implemented by resource managers
* AbiPy analyzes *stderr* files and uses heuristic rules to detect this kind of problem

## Netcdf files in Abinit

#### Philosophy

* A *ncfile* is a container of "objects"
* Each object implements the *ncwrite* method to dump its status
* Client code uses these methods to implement *composition*
* A *GSR.nc* file, for instance, has a *crystal*, an *header*, *ebands* and *results_gs*

#### Fortran code for GSR.nc
<p></p>

```Fortran
ncerr = nctk_open_create(ncid, "si_scf_GSR.nc", xmpi_comm_self)
! Write hdr, crystal and band structure
ncerr = hdr%ncwrite(ncid, fform_den, nc_define=.True.)
ncerr = crystal%ncwrite(ncid)
ncerr = ebands%ncwrite(ncid)
! Add energy, forces, stresses
ncerr = results_gs%ncwrite(ncid, dtset%ecut, dtset%pawecutdg)
ncerr = nf90_close(ncid)
```
#### The same pattern can be used for other files:
<p></p>

```fortran
NCF_CHECK(nctk_open_create(ncid, "out_PHDOS.nc", xmpi_comm_self))
! PHDOS.nc has a crystalline structure + DOS values
NCF_CHECK(cryst%ncwrite(ncid))
NCF_CHECK(phdos%ncwrite(ncid)
```

#### Advantages of the *ncwrite* API:

* Code reuse. Need to implement extra logic only for *extra data*
* Standardized format for *hdr*, *crystal*, *ebands* ...
* Post-processing tools need metadata (*e.g.* *crystal*) to interpret the data 
* Less boiler-plate code to implement parsers and post-processing tools

#### Abipy implementation based on mixins:
<p></p>

```python
class GsrFile(AbinitNcFile, Has_Header, Has_Structure, Has_ElectronBands):
    """This file contains ground-state results"""
    
class FatBandsFile(AbinitNcFile, Has_Header, Has_Structure, Has_ElectronBands):
    """This file contains LM-projected bands"""
```

#### Easy-to-use API that plays well with the dynamic nature of the python language
<p></p>

```python
for path in ("out_GSR.nc", "out_FATBANDS.nc", "out_WFK.nc", "out_SIGEPH.nc"):
    abifile = abilab.abiopen(path)
    abifile.ebands.plot()
```

In [38]:
gsr = abilab.abiopen("si_scf_GSR.nc")
for key in ("npwarr", "pspso", "ecutsm", "codvsn"):
    print(key, gsr.hdr[key])

npwarr [181 178 193 195 180 177 193 196 193 189 193 186 202 195 192 190 192 198
 198 194 190 194 195 194 197 198 197 196 200]
pspso [0]
ecutsm 0.0
codvsn 8.0.6


In [39]:
gsr.energy_terms

Term                  Value
e_localpsp            -68.62575616682041 eV
e_eigenvalues         4.453251781809304 eV
e_ewald               -226.94266538122858 eV
e_hartree             14.989583868349118 eV
e_corepsp             2.262480246606245 eV
e_corepspdc           0.0 eV
e_kinetic             80.6603511758275 eV
e_nonlocalpsp         52.15353045099117 eV
e_entropy             0.0 eV
entropy               0.0 eV
e_xc                  -95.73399250916707 eV
e_xcdc                0.0 eV
e_paw                 0.0 eV
e_pawdc               0.0 eV
e_elecfield           0.0 eV
e_magfield            0.0 eV
e_fermie              5.5984532787385985 eV
e_sicdc               0.0 eV
e_exactX              0.0 eV
h0                    0.0 eV
e_electronpositron    0.0 eV
edc_electronpositron  0.0 eV
e0_electronpositron   0.0 eV
e_monopole            0.0 eV

<center><img src="./assets/technical_problems.jpg" width="85%" align="center"></center>
<br><hr>

* Restart capabilities and IO
* Extending *TesSuite* with AbiPy
* *TestSuite* is too gentle but clusters are dark and full of errors 🐉🐉🐉
* Using AbiPy to benchmark ABINIT

## Restart capabilities and IO

#### Clean exit with *--timelimit* and smart-io mode (prtwf -1)
<p></p>

```sh
    $ cat job.sh

    #SBATCH --time=12:00:00
    abinit --timelimit 12:00:00 < run.file > run.log 2> run.err
```

* Available for iterative algorithms (*Relax*, *Scf*)
* Compute wall-time required by one iteration, producd *restart files* if *timelimit* is approaching
* *prtwf* -1: Print WFK only if *not converged* to reduce IO-pressure and avoid disk-quota
* AbiPy will restart the job from the previous results 

#### Restart from *netcdf* file

* EPH approach: Save results to SIGEPH with table *done(ikpt, spin)*

#### Unfortunately:

* Not all the drivers support restart e.g. GW or BSE
* Output of certain files may use smart-io mode by default *e.g.* first-order WFK
* The number of files *per user* is limited. Should make list of files that may be removed e.g. *_OUT.nc* 

## Extending TestSuite with AbiPy

* More that 1200 tests in *TestSuite* still the infrastructure <u>is not bulletproof</u>
* We assume the reference results are *always* correct but
     * mistakes may happen when updating refs
     * in comes cases, inconsistencies can be spotted by analyzing the raw data

#### Based on a true story:

* A bug in the calculation of the atom-projected phonon DOS entered trunk/develop
* All the refs were updated so buildbot was happy!
* I had a look at the diffs but it was like finding needle in the haystack
* Fortunately, AbiPy detected the problem because there was a test based 
  on the post-processing of the numerical results...

```python
# Test whether projected DOSes computed by anaddb integrate to 3*natom.
ddb = abilab.abiopen("out_DDB")

for dos_method in ("tetra", "gaussian"):
    # Get phonon bands and dos with anaddb.
    phbst_nc, phdos_nc = ddb.anaget_phbst_and_phdos_files(dos_method=dos_method)
    phbands, phdos = phbst_nc.phbands, phdos_nc.phdos

    # Total PHDOS should integrate to 3 * natom
    assert_almost_equal(phdos.integral_value, len(phbands.structure) * 3)

    # Summing projected DOSes over types should give the total DOS.
    pj_sum = sum(pjdos.integral_value for pjdos in phdos_file.pjdos_symbol.values())
    assert_almost_equal(phdos.integral_value, pj_sum)

    # Summing proj-DOSes over types and directions should give the total DOS.
    values = phdos_file.reader.read_value("pjdos_rc_type").sum(axis=(0, 1))
    tot_dos = abilab.Function1D(phdos.mesh, values)
    assert_almost_equal(phdos.integral_value, tot_dos.integral_value)
```

## Want to run the test in parallel with MPI? Just add:
<p></p>

```python
for mpi_procs in range(1, 10**23):
    ddb.anaget_phbst_and_phdos_files(dos_method=dos_method, mpi_procs=mpi_procs)
```

* The idea is not new: FFTW3 built-in tests use this kind of internal consistency check
* Implementing this kind of tests in Fortran is a real pain (I did it for the FFT machinery in *fftprof*)
* python and the AbiPy API represent a very flexible approach to implement this kind of agnostic tests
* Want to run the test in parallel with MPI? Just add:
<p></p>

```python
for mpi_procs in range(1, 10**23):
    ddb.anaget_phbst_and_phdos_files(dos_method=dos_method, mpi_procs=mpi_procs)
```

### TestSuite is too gentle but clusters are dark and full of errors  🐉🐉🐉

> Stress testing involves testing beyond normal operational capacity, 
> often to a breaking point in order to:
>    - determine breaking points or safe usage limits
>    - determine how exactly a system fails
>
>  (*wikipedia*)

#### Questions:

* How many tests do we perform before saying that one algo is clearly superior to another one?
* Do we have a suite of input systems designed to reveal weaknesses in algorithms?
* Is it possible to handle some of these errors at the Fortran level?

#### Stress testing is not easy because one needs logic to cope with errors 

* AbiPy uses integration tests to check *ErrorHandlers* in which the code is supposed to 
  **fail several times** before giving the right answer.
* I'm not claiming that stress testing with AbiPy is gonna be easy, but it's clear that 
  *TestSuite* was not designed with this goal in mind.  

#### Using AbiPy to benchmark ABINIT

* The programmatic API allows one to write benchmarks in a few lines of code
* Want to analyze the strong scaling efficiency of the GS solver with different configurations 
  of MPI/OMP and input variables?

```python
# Titanium with 256 atoms and k-point sampling.
# GS calculations with paral_kgb == 1 and different values of wfoptalg.

# List of MPI distribution.
pconfs = [
     dict(npkpt=2, npband=8 , npfft=8 ),  # 128
     dict(npkpt=2, npband=8 , npfft=16),  # 256
     dict(npkpt=2, npband=16, npfft=16),  # 512
]

omp_list = [1, 2, 4] # List of OpenMP threads

flow = BenchmarkFlow()
template = generate_input()

for wfoptalg in [1, 10, 14]:
    work = flowtk.Work()
    for d, omp_threads in product(pconfs, omp_list):
        mpi_procs = reduce(operator.mul, d.values(), 1)
        manager = manager.new_with_fixed_mpi_omp(mpi_procs, omp_threads)
        inp = template.new_with_vars(d, wfoptalg=wfoptalg)
        work.register_scf_task(inp, manager=manager)
    flow.register_work(work)
flow.allocate()
```

## Conclusions

* This talk has no conclusions because we have not yet found optimal solutions to the problems 
  mentioned in the previous slides except one:
  
#### Different Problems Require Different Solutions Based On Different Technologies